## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

4891it [02:37, 31.00it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 4891


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 4891/4891 [00:21<00:00, 223.13it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,InvestmentCanvas,Berlin,Stop listening to your bank ❌👂🏦. Use our free ...,2019-01-30 18:40:17,249,47,12,False,2020-08-10 16:28:02,S&amp;P 500 is back at pre-corona highs.📉📈 Do ...,"[COVID19, investing]",Buffer,False
0,April Thompson,,Married mother of 2 beautiful bad ass women & ...,2009-09-05 04:46:50,1031,1250,3356,False,2020-08-10 16:28:02,In case you needed a little chuckle this Monda...,"[kayleighmcenany, CuomoPrimeTime, COVID19]",Twitter for Android,False
0,Institut Montaigne_EN,"Paris, France",French nonpartisan & nonprofit #thinktank comm...,2017-10-10 08:47:22,26164,296,3306,False,2020-08-10 16:28:01,"🦠 #Covid19: ""#Psychiatry is in a permanent st...","[Covid19, Psychiatry]",Sprout Social,False
0,DC Health,District of Columbia,"The DC Department of Health promotes health, w...",2010-02-26 16:50:13,10074,4252,667,True,2020-08-10 16:28:00,Updated Ward-level data for positive coronavir...,[COVID19],Twitter Web App,False
0,Hamzat Lawal💡Hamzy! {Etim} 🔦,Village Square,Grassroots @CODEActivist Campaigner leading @C...,2010-07-26 09:13:51,100580,7242,26320,True,2020-08-10 16:27:59,While conscious of the exposures of being on g...,None,Twitter for iPhone,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (97710, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 4891 past tweets: 97710 all tweets: 102601


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (102601, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)